# SISR with Plain DNN

There are amny ways to construct a deep neural network. The following figure illustrate the structures of VGG-19, AlexNet, and ResNet:

<img src='images/resnet.png' />

In this example, we test SISR with a plain-convolutional-neural-network (VGG-like)



In [11]:
import os, csv, logging, argparse, glob, h5py, pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Activation
from tensorflow.keras.layers import Conv2D, BatchNormalization, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError, MeanSquaredError
#-----------------------------------------------------------------------
# Utility Functions
#-----------------------------------------------------------------------
# Load input/output data for model
def loadIOTab(srcx, srcy, dropna=False):
    import pandas as pd
    # Scan for input data
    logging.info("Reading input X from: "+ srcx)
    xfiles = []
    for root, dirs, files in os.walk(srcx): 
        for fn in files: 
            if fn.endswith('.npy'): 
                 xfiles.append({'date':fn.replace('.npy',''), 'xuri':os.path.join(root, fn)})
    xfiles = pd.DataFrame(xfiles)
    print("... read input size: "+str(xfiles.shape))
    # Scan for input data
    logging.info("Reading output Y from: "+ srcy)
    yfiles = []
    for root, dirs, files in os.walk(srcy): 
        for fn in files: 
            if fn.endswith('.npy'): 
                 yfiles.append({'date':fn.replace('.npy',''), 'yuri':os.path.join(root, fn)})
    yfiles = pd.DataFrame(yfiles)
    print("... read output size: "+str(yfiles.shape))
    # Create complete IO-data
    print("Merge input/output data.")
    iotab = pd.merge(yfiles, xfiles, on='date', sort=True)
    print("... data size after merging: "+str(iotab.shape))
    # Done
    return(iotab)

def load_sprec(flist):
    ''' Load a list a Surface Precipitation files (in npy format) into one numpy array. '''
    xdata = []
    for f in flist:
        tmp = np.load(f)                            # Load numpy array
        xdata.append(np.expand_dims(tmp, axis=2))   # Append axis to the end
    x = np.array(xdata, dtype=np.float32)
    return(x)

def data_generator(iotab, batch_size):
    ''' Data generator for batched processing. '''
    nSample = len(iotab)
    # This line is just to make the generator infinite, keras needs that
    while True:
        # Initialize the sample counter
        batch_start = 0
        batch_end = batch_size
        while batch_start < nSample:
            limit = min(batch_end, nSample)                     # Correct the end-pointer for the final batch
            X = load_sprec(iotab['xuri'][batch_start:limit])    # Load X
            Y = load_sprec(iotab['yuri'][batch_start:limit])    # Load Y
            yield (X,Y)                                         # Send a tuple with two numpy arrays
            batch_start += batch_size   
            batch_end += batch_size
    # End of generator

# Function to give report
def report_sisr(y_true, y_pred):
    import pandas as pd
    import sklearn.metrics as metrics
    output = []
    # Loop through samples
    n, h, w, c = y_true.shape
    for i in range(n):
        yt = y_true[i,:,:,:].flatten()
        yp = y_pred[i,:,:,:].flatten()
        # Calculate measures
        results = {}
        results['y_true_mean'] = yt.mean()
        results['y_true_var'] = yt.var()
        results['y_pred_mean'] = yp.mean()
        results['y_pred_var'] = yp.var()
        results['rmse'] = np.sqrt(metrics.mean_squared_error(yt,yp))
        if y_pred.var()<=10e-8:
            results['corr'] = 0
        else:
            results['corr'] = np.corrcoef(yt,yp)[0,1]
        output.append(results)
    # Return results
    return(pd.DataFrame(output))

# Create cross validation splits
def create_splits(iotable, prop=0.2):
    idxes = np.arange(iotable.shape[0])     # Create indexes
    idxes = np.random.permutation(idxes)    # Permute indexes
    idx_break = int(len(idxes)*prop)        # Index for the split point
    idx_test = idxes[:idx_break]
    idx_train = idxes[idx_break:]
    return((idx_train, idx_test))

#-----------------------------------------------------------------------
# The model
#-----------------------------------------------------------------------
def init_model_plaindnn(input_shape):
    """
    :Return: 
      Newly initialized model for image up-scaling.
    :param 
      int input_shape: The number of variables to use as input features.
    """
    # Input layer
    inputs = Input(shape=input_shape)
    # blovk1: CONV -> CONV
    x = BatchNormalization(axis=-1)(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='conv1', padding='same')(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='conv2', padding='same')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='conv3', padding='same')(x)
    # Output block: UP_SAMPLE -> CONV
    x = UpSampling2D((2, 2), name='upsampple')(x)
    x = Conv2D(filters=1, kernel_size=(3,3), activation='relu', name='conv4', padding='same')(x)
    out = BatchNormalization(axis=-1)(x)
    # Initialize model
    model = Model(inputs = inputs, outputs = out)
    # Define compile parameters
    adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='mae', optimizer=adam, metrics=['mse','cosine_similarity'])
    return(model)

In [2]:
# Set up parameters
#DIRORI = 'D:\\data\\vvm_sprec\\original\\'
#DIR2nd = 'D:\\data\\vvm_sprec\\scale_0.5\\'
#DIR4th = 'D:\\data\\vvm_sprec\\scale_0.25\\'
#DIR8th = 'D:\\data\\vvm_sprec\\scale_0.125\\'
DIRORI = '/home/tsyo/sisrdata/original/'
DIR2nd = '/home/tsyo/sisrdata/scale_0.5/'
DIR4th = '/home/tsyo/sisrdata/scale_0.25/'
DIR8th = '/home/tsyo/sisrdata/scale_0.125/'

import numpy as np
import logging, os
import joblib

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [12]:
iotab = loadIOTab(DIR8th, DIR4th, dropna=True)
idx_trains, idx_tests = create_splits(iotab)
model = init_model_plaindnn((128, 128, 1))
model.summary()

steps_train = np.ceil(len(idx_trains)/64)
steps_test = np.ceil(len(idx_tests)/64)

INFO:root:Reading input X from: /home/tsyo/sisrdata/scale_0.125/
INFO:root:Reading output Y from: /home/tsyo/sisrdata/scale_0.25/


... read input size: (2585, 2)
... read output size: (2585, 2)
Merge input/output data.
... data size after merging: (2585, 3)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 128, 1)       4         
_________________________________________________________________
conv1 (Conv2D)               (None, 128, 128, 64)      640       
_________________________________________________________________
batch_normalization_6 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
conv2 (Conv2D)               (None, 128, 128, 64)      36928     
_________________________________________________________________
conv3 (Conv2D)               (None, 128, 128, 64)      36928    

In [13]:
hist = model.fit_generator(data_generator(iotab.iloc[idx_trains,:], 64), steps_per_epoch=steps_train, epochs=1, max_queue_size=64, verbose=1)

33/33 [==============================] - 394s 12s/step - loss: 0.0470 - mse: 0.7474 - cosine_similarity: 0.0016


In [5]:
print(hist.history)

{'loss': [0.4251700116887083, 0.07193213505976666, 0.04817474693982237, 0.047290743312418344, 0.04804149404081662, 0.04825442931407656, 0.047824668973626666, 0.04770570989100108, 0.046991414111287724, 0.04702789577109671], 'mse': [1.4727378, 0.73468393, 0.7347262, 0.7347137, 0.73467815, 0.7346771, 0.734693, 0.73467344, 0.7347365, 0.73474807], 'cosine_similarity': [0.017761327, 0.08921894, 0.045139316, 0.046012778, 0.04570692, 0.05332372, 0.046766564, 0.07473871, 0.07242183, 0.073652536]}


In [7]:
import h5py, json
model.save_weights('data/plain_dnn_weights.h5')
with open('data/plain_dnn_model.json', 'w') as f:
    json.dump(model.to_json(), f)

In [15]:
y_pred = model.predict_generator(data_generator(iotab.iloc[idx_tests,:], 64), steps=steps_test, max_queue_size=64, verbose=1)

9/9 [==============================] - 10s 1s/step


In [16]:
y_true = load_sprec(iotab['yuri'].iloc[idx_tests])
print(y_true.shape)
print(y_pred.shape)

(517, 256, 256, 1)
(517, 256, 256, 1)


In [17]:
report = report_sisr(y_true, y_pred)
report.head()

,y_true_mean,y_true_var,y_pred_mean,y_pred_var,rmse,corr
0,0.008202,0.013276,0.0,0.0,0.115512,0
1,0.194439,2.504146,0.0,0.0,1.594350,0
2,0.000258,0.000066,0.0,0.0,0.008155,0
3,0.076589,1.409674,0.0,0.0,1.189764,0
4,0.031227,0.316459,0.0,0.0,0.563413,0


In [17]:
report.to_csv('data/test.csv')